# **Установка нужных версий для работы**

In [ ]:
!pip install polars==0.20.19 scipy==1.13.0 scikit-learn==1.4.1.post1 numpy==1.26.4 pandas==2.0.3 rapidfuzz~=3.8.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 31.6 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 0.20.2
    Uninstalling polars-0.20.2:
      Successfully uninstalled polars-0.20.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfu

# **Импорты**

In [ ]:
import polars as pl
import numpy as np
import pandas as pd
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from rapidfuzz import process
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# **Директория с датасетом**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATASET_DIR = 'drive/MyDrive/dataset'

# **Простой алгоритм**

In [ ]:
def weighted_rating(count, avg, quantile: pl.DataFrame, mean: pl.DataFrame):
    quantile = float(quantile.head(1).row(0)[0])
    return (count / (count + quantile) * avg) + (quantile / (quantile + count) * mean)

In [ ]:
def get_content_simple_recs(movies_metadata: pl.DataFrame, head_count: int) -> pl.DataFrame:
    """
    Функция, возвращающие общие рекомендации по фильмам, без уточняющих характеристик.
    :param movies_metadata:
    :param head_count: Количество фильмов, которые надо возвратить в итоговом дата фрейме
    :return:
    """
    # Получаем среднее значение оценок под фильмами
    mean: pl.DataFrame = movies_metadata.select('vote_average').mean()
    # Агрегируем столбцы этого DataFrame до квантиля 0.90.
    quantile: pl.DataFrame = movies_metadata.select('vote_count').quantile(0.90)
    # Получаем дата фрейм из значений количества голосов больше квантиля
    q_movies: pl.DataFrame = movies_metadata.filter(pl.col('vote_count') >= quantile)
    # Дописываем столбец со взвешенным рейтингом, название столбца - 'score'
    q_movies: pl.DataFrame = q_movies.with_columns(
        weighted_rating(
            pl.col('vote_count'),
            pl.col('vote_average'),
            quantile,
            mean
        ).alias('Оценка алгоритма'),
        pl.col('title').alias('Название'),
        pl.col('vote_count').alias('Количество оценок'),
        pl.col('vote_average').alias('Средняя оценка')
    )
    # Сортируем дата фрейм по убыванию взвешенного рейтинга
    q_movies: pl.DataFrame = q_movies.sort('Оценка алгоритма', descending=True)
    return q_movies[['Название', 'Количество оценок', 'Средняя оценка', 'Оценка алгоритма']].head(head_count)

# **По содержанию**

# TF-IDF

In [ ]:
def get_content_tfidf_recs(movies_metadata: pl.DataFrame, title: str) -> pl.DataFrame:
    tfidf: TfidfVectorizer = TfidfVectorizer(stop_words='english')
    movies_metadata: pl.DataFrame = movies_metadata.with_columns(
        pl.col('overview').fill_null('')
    ).drop('vote_average', 'vote_count')
    overview_series: pl.Series = movies_metadata.select('overview').to_series()

    # Составляем матрицу TF-IDF
    from scipy.sparse import csr_matrix
    tfidf_matrix: csr_matrix = tfidf.fit_transform(overview_series)
    cosine_sim: np.ndarray = linear_kernel(tfidf_matrix, tfidf_matrix)
    movies_metadata = movies_metadata.with_row_index("index")
    # Получаем индекс фильма, название которого совпадает с заданным
    expr: pl.Expr = pl.all_horizontal(
        pl.col('title') == title
    )
    idx = movies_metadata.row(by_predicate=expr, named=True)['index']
    # Получаем попарную схожесть всех фильмов с фильмом, который нам дан
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Сортируем фильмы на основании очков схожести
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Получаем очки для 10 самых похожих фильмов
    sim_scores = sim_scores[1:11]

    # Получаем индексы фильмов
    movie_indices = [i[0] for i in sim_scores]
    return movies_metadata.select('title')[movie_indices]

# По ключевым словам

In [ ]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan


def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        if len(names) > 3:
            names = names[:3]
        return names

    return []


def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        # Проверяем что директор существует. Если нет, возвращаем пустую строку
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''


def get_recommendations(title, metadata: pd.DataFrame, indices, cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return metadata['title'].iloc[movie_indices]


def create_soup(x):
    return ' '.join(str(x['keywords'])) + ' ' + ' '.join(str(x['cast'])) + ' ' + str(x['director']) + ' ' + ' '.join(str(x['genres']))


def get_content_keywords_recs(
        title: str,
        movies_df,
        credits,
        keywords
) -> pd.DataFrame:
    metadata = movies_df
    # Сломанные записи
    if len(metadata) > 33000:
      metadata = metadata.drop(35587)
    if len(metadata) > 25000:
      metadata = metadata.drop(29503)
    if len(metadata) > 15000:
      metadata = metadata.drop(19730)
    keywords['id'] = keywords['id'].astype('int')
    credits['id'] = credits['id'].astype('int')
    metadata['id'] = metadata['id'].astype('int')
    metadata = metadata.merge(credits, on='id')
    metadata = metadata.merge(keywords, on='id')

    features = ['cast', 'crew', 'keywords', 'genres']

    for feature in features:
        metadata[feature] = metadata[feature].apply(literal_eval)
    metadata['director'] = metadata['crew'].apply(get_director)
    for feature in features:
        metadata[feature] = metadata[feature].apply(get_list)
    for feature in features:
        metadata[feature] = metadata[feature].apply(clean_data)
    metadata['soup'] = metadata.apply(create_soup, axis=1)
    count = CountVectorizer(stop_words='english')
    count_matrix = count.fit_transform(metadata['soup'])
    cosine_sim = cosine_similarity(count_matrix, count_matrix)
    metadata = metadata.reset_index()
    indices = pd.Series(metadata.index, index=metadata['title'])
    return get_recommendations(title, metadata, indices, cosine_sim)

# **Коллаборативная фильтрация**

# K ближайших соседей

In [ ]:
def get_collaborative_knn_recs(movie_names: pd.DataFrame, ratings_data: pd.DataFrame, movie_name: str):
    # Дата фрейм с названием фильма и его жанром
    movie_names = movie_names[['title', 'genres']]
    # Дата фрейм, в котором
    movies_users: pd.DataFrame = ratings_data.pivot(index=['userId'], columns=['movieId'], values='rating').fillna(0)
    # Преобразовываем в разреженную матрицу (CSR)
    mat_movies_users: csr_matrix = csr_matrix(movies_users.values)
    model_knn: NearestNeighbors = NearestNeighbors(metric='cosine', algorithm='auto', n_neighbors=30, n_jobs=-1)
    model_knn.fit(mat_movies_users)
    movie_index: int = process.extractOne(movie_name, movie_names['title'])[2]
    distances, indices = model_knn.kneighbors(mat_movies_users[movie_index], n_neighbors=20)
    recc_movie_indices: list = sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())),
                                      key=lambda x: x[1])[:0:-1]
    # Список с рекомендациями
    recommend_list = []
    # На каждый индекс рекомендаций
    for val in recc_movie_indices:
        # Добавляем в датафрейм рекомендаций названий фильма и расстояние
        recommend_list.append({'Title': movie_names['title'][val[0]], 'Distance': val[1]})
    # Датафрейм с рекомендациями
    df = pd.DataFrame(recommend_list, index=range(1, 20))
    return df

# SVD

In [ ]:
def get_collaborative_svd_recs(user_id: int, num_recommendations: int, movies_df, ratings_df):
    # Merge the two datasets
    #movies_df.index = movies_df['movieId']
    #ratings_df.index = ratings_df['movieId']
    df: pd.DataFrame = pd.merge(ratings_df, movies_df, on="movieId")
    pivot_table: pd.DataFrame = df.pivot_table(index="userId", columns="title", values="rating")
    pivot_table = pivot_table.dropna(axis='columns', thresh=2)
    overall_mean = pivot_table.mean(axis=1)
    # Fill missing values with the mean rating
    pivot_table.fillna(overall_mean, axis='index', inplace=True)
    from scipy.sparse.linalg import svds
    pivot_np = pivot_table.to_numpy(na_value=2.5)
    U, sigma, Vt = svds(pivot_np, k=1)
    user_item_matrix = sigma * Vt.T
    user_rating_vector = user_item_matrix[user_id - 1]
    similarity_scores = np.corrcoef(user_rating_vector, user_item_matrix)[0, 1:]
    top_movies = np.argsort(similarity_scores)[::-1][:num_recommendations]
    return movies_df.iloc[top_movies]["title"]

# **Песочница**

In [ ]:
import timeit

In [ ]:
movies_df: pl.DataFrame = (pl.read_csv(f'{DATASET_DIR}/movies_metadata.csv', infer_schema_length=100000)
                               .select(pl.col("id", "title", "overview", "vote_average", "vote_count", "genres"))).head(30000)

In [ ]:
user_ratings_df: pl.DataFrame = (pl.read_csv(f'{DATASET_DIR}/ratings_small.csv')
                                     .select(pl.col("userId", "movieId", "rating"))).head(80000)

In [ ]:
print("---Время работы простейшего алгоритма---")

times = 4000
time = timeit.timeit(lambda: get_content_simple_recs(movies_df, 10), number=times)
print(time/times, " секунд ушло в среднем на 1 из ", times, " прогонов")
print(time, " секунд всего ушло на ", times, " прогонов")

time = timeit.timeit(lambda: get_content_simple_recs(movies_df.head(25000), 10), number=times)
print(time/times, " секунд ушло в среднем на 1 из ", times, " прогонов")
print(time, " секунд всего ушло на ", times, " прогонов")

time = timeit.timeit(lambda: get_content_simple_recs(movies_df.head(20000), 10), number=times)
print(time/times, " секунд ушло в среднем на 1 из ", times, " прогонов")
print(time, " секунд всего ушло на ", times, " прогонов")

time = timeit.timeit(lambda: get_content_simple_recs(movies_df.head(15000), 10), number=times)
print(time/times, " секунд ушло в среднем на 1 из ", times, " прогонов")
print(time, " секунд всего ушло на ", times, " прогонов")

time = timeit.timeit(lambda: get_content_simple_recs(movies_df.head(10000), 10), number=times)
print(time/times, " секунд ушло в среднем на 1 из ", times, " прогонов")
print(time, " секунд всего ушло на ", times, " прогонов")

time = timeit.timeit(lambda: get_content_simple_recs(movies_df.head(5000), 10), number=times)
print(time/times, " секунд ушло в среднем на 1 из ", times, " прогонов")
print(time, " секунд всего ушло на ", times, " прогонов")

---Время работы простейшего алгоритма---
0.003563161302499992  секунд ушло в среднем на 1 из  4000  прогонов
14.252645209999969  секунд всего ушло на  4000  прогонов
0.0031699928517499956  секунд ушло в среднем на 1 из  4000  прогонов
12.679971406999982  секунд всего ушло на  4000  прогонов
0.0027842177917500096  секунд ушло в среднем на 1 из  4000  прогонов
11.136871167000038  секунд всего ушло на  4000  прогонов
0.002161087751750003  секунд ушло в среднем на 1 из  4000  прогонов
8.644351007000012  секунд всего ушло на  4000  прогонов
0.0014267223905000037  секунд ушло в среднем на 1 из  4000  прогонов
5.706889562000015  секунд всего ушло на  4000  прогонов
0.0015273430755000134  секунд ушло в среднем на 1 из  4000  прогонов
6.109372302000054  секунд всего ушло на  4000  прогонов


In [ ]:
print("---Время работы алгоритма на базе TF-IDF---")
times = 2
time = timeit.timeit(lambda: get_content_tfidf_recs(movies_df.head(30000), 'Jumanji').head(10), number=times)
print(time, " секунд всего ушло на ", times, " прогонов с размером датасета")

time = timeit.timeit(lambda: get_content_tfidf_recs(movies_df.head(25000), 'Jumanji').head(10), number=times)
print(time, " секунд всего ушло на ", times, " прогонов")

time = timeit.timeit(lambda: get_content_tfidf_recs(movies_df.head(20000), 'Jumanji').head(10), number=times)
print(time, " секунд всего ушло на ", times, " прогонов")

time = timeit.timeit(lambda: get_content_tfidf_recs(movies_df.head(15000), 'Jumanji').head(10), number=times)
print(time, " секунд всего ушло на ", times, " прогонов")

time = timeit.timeit(lambda: get_content_tfidf_recs(movies_df.head(10000), 'Jumanji').head(10), number=times)
print(time, " секунд всего ушло на ", times, " прогонов")

time = timeit.timeit(lambda: get_content_tfidf_recs(movies_df.head(5000), 'Jumanji').head(10), number=times)
print(time, " секунд всего ушло на ", times, " прогонов")

#time = timeit.timeit(lambda: get_content_simple_recs(movies_df.head(5000), 10), number=times)
#print(time/times, " секунд ушло в среднем на 1 из ", times, " прогонов")
#print(time, " секунд всего ушло на ", times, " прогонов")

---Время работы алгоритма на базе TF-IDF---
41.139004016999934  секунд всего ушло на  2  прогонов с размером датасета
24.820393908999904  секунд всего ушло на  2  прогонов
17.50840908100008  секунд всего ушло на  2  прогонов
10.353564688000006  секунд всего ушло на  2  прогонов
4.459642247999909  секунд всего ушло на  2  прогонов
1.4369325990001016  секунд всего ушло на  2  прогонов


In [ ]:
print("---Время работы алгоритма на базе заготовленных ключевых слов---")
times = 4
movies_pd = movies_df.to_pandas()
credits = pd.read_csv(f'{DATASET_DIR}/credits.csv', nrows=15000)
keywords = pd.read_csv(f'{DATASET_DIR}/keywords.csv')

pd.options.mode.chained_assignment = None

time = timeit.timeit(lambda: get_content_keywords_recs('Jumanji', movies_pd.head(30000), credits, keywords).head(10), number=times)
print(time, " секунд всего ушло на ", times, " прогонов")

time = timeit.timeit(lambda: get_content_keywords_recs('Jumanji', movies_pd.head(25000), credits, keywords).head(10), number=times)
print(time, " секунд всего ушло на ", times, " прогонов")

time = timeit.timeit(lambda: get_content_keywords_recs('Jumanji', movies_pd.head(20000), credits, keywords).head(10), number=times)
print(time, " секунд всего ушло на ", times, " прогонов")

time = timeit.timeit(lambda: get_content_keywords_recs('Jumanji', movies_pd.head(15000), credits, keywords).head(10), number=times)
print(time, " секунд всего ушло на ", times, " прогонов")

time = timeit.timeit(lambda: get_content_keywords_recs('Jumanji', movies_pd.head(10000), credits, keywords).head(10), number=times)
print(time, " секунд всего ушло на ", times, " прогонов")

time = timeit.timeit(lambda: get_content_keywords_recs('Jumanji', movies_pd.head(5000), credits, keywords).head(10), number=times)
print(time, " секунд всего ушло на ", times, " прогонов")

---Время работы алгоритма на базе заготовленных ключевых слов---
85.72211765100019  секунд всего ушло на  4  прогонов
89.46958638199976  секунд всего ушло на  4  прогонов
82.16336677299932  секунд всего ушло на  4  прогонов
80.80789647300026  секунд всего ушло на  4  прогонов
56.522462799000095  секунд всего ушло на  4  прогонов
29.150475607999397  секунд всего ушло на  4  прогонов


In [ ]:
print("---Время работы алгоритма KNN ---")
movies_pd = movies_df.to_pandas()
user_ratings_pd = user_ratings_df.to_pandas()
times = 100

time = timeit.timeit(lambda: get_collaborative_knn_recs(movies_pd.head(30000), user_ratings_pd, 'Jumanji').iloc[::-1], number=times)
print(time, " секунд всего ушло на ", times, " прогонов")

time = timeit.timeit(lambda: get_collaborative_knn_recs(movies_pd.head(25000), user_ratings_pd, 'Jumanji').iloc[::-1], number=times)
print(time, " секунд всего ушло на ", times, " прогонов")

time = timeit.timeit(lambda: get_collaborative_knn_recs(movies_pd.head(20000), user_ratings_pd, 'Jumanji').iloc[::-1], number=times)
print(time, " секунд всего ушло на ", times, " прогонов")

time = timeit.timeit(lambda: get_collaborative_knn_recs(movies_pd.head(15000), user_ratings_pd, 'Jumanji').iloc[::-1], number=times)
print(time, " секунд всего ушло на ", times, " прогонов")

time = timeit.timeit(lambda: get_collaborative_knn_recs(movies_pd.head(10000), user_ratings_pd, 'Jumanji').iloc[::-1], number=times)
print(time, " секунд всего ушло на ", times, " прогонов")

time = timeit.timeit(lambda: get_collaborative_knn_recs(movies_pd.head(5000), user_ratings_pd, 'Jumanji').iloc[::-1], number=times)
print(time, " секунд всего ушло на ", times, " прогонов")

---Время работы алгоритма KNN ---
13.619223024999883  секунд всего ушло на  100  прогонов
13.17771265600004  секунд всего ушло на  100  прогонов
13.02451474999998  секунд всего ушло на  100  прогонов
13.228877860000466  секунд всего ушло на  100  прогонов
13.447778724000273  секунд всего ушло на  100  прогонов
13.305383921000612  секунд всего ушло на  100  прогонов


In [ ]:
print("---Время работы алгоритма SVD ---")
times = 100
movies_df: pd.DataFrame = pd.read_csv(f'{DATASET_DIR}/movies_metadata.csv', low_memory=True, encoding='latin-1')
movies_df.drop(columns=['budget'])
movies_df = movies_df.drop([19730, 29503]).head(30000)
movies_df.rename(columns={'id': 'movieId'}, inplace=True)
ratings_df: pd.DataFrame = pd.read_csv(f'{DATASET_DIR}/ratings.csv', encoding='latin-1').head(80000)
ratings_df['movieId'] = ratings_df['movieId'].astype(int)
movies_df['movieId'] = movies_df['movieId'].astype(int)
import warnings
warnings.filterwarnings("ignore")


time = timeit.timeit(lambda: get_collaborative_svd_recs(1, 20, movies_df, ratings_df), number=times)
print(time, " секунд всего ушло на ", times, " прогонов")

time = timeit.timeit(lambda: get_collaborative_svd_recs(1, 20, movies_df.head(25000), ratings_df), number=times)
print(time, " секунд всего ушло на ", times, " прогонов")

time = timeit.timeit(lambda: get_collaborative_svd_recs(1, 20, movies_df.head(20000), ratings_df), number=times)
print(time, " секунд всего ушло на ", times, " прогонов")

time = timeit.timeit(lambda: get_collaborative_svd_recs(1, 20, movies_df.head(15000), ratings_df), number=times)
print(time, " секунд всего ушло на ", times, " прогонов")

time = timeit.timeit(lambda: get_collaborative_svd_recs(1, 20, movies_df.head(10000), ratings_df), number=times)
print(time, " секунд всего ушло на ", times, " прогонов")

time = timeit.timeit(lambda: get_collaborative_svd_recs(1, 20, movies_df.head(5000), ratings_df), number=times)
print(time, " секунд всего ушло на ", times, " прогонов")

---Время работы алгоритма SVD ---
29.31651280299957  секунд всего ушло на  100  прогонов
21.2284621889994  секунд всего ушло на  100  прогонов
22.004887082001005  секунд всего ушло на  100  прогонов
19.33256030300072  секунд всего ушло на  100  прогонов
15.2411935300006  секунд всего ушло на  100  прогонов
11.502154998999686  секунд всего ушло на  100  прогонов
